In [289]:
import re
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

In [290]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

In [291]:
text

'Вы  лучший ответ на проблемы которые возникли в понедельникДумайте позитивно и верьте в свою способность достигать отличных результатовЕсли вы смогли в понедельник подняться с постели значит вы супер герой'

In [292]:
len(text)

205

In [293]:
num_characters = 34 #33 буквы + пробел

In [294]:
import keras
from keras.preprocessing.text import Tokenizer

In [295]:
tokenizer = Tokenizer(num_words=num_characters, char_level=True)

In [296]:
tokenizer.fit_on_texts(text)

In [297]:
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'т': 3, 'е': 4, 'и': 5, 'в': 6, 'н': 7, 'с': 8, 'л': 9, 'п': 10, 'ь': 11, 'ы': 12, 'р': 13, 'а': 14, 'д': 15, 'у': 16, 'к': 17, 'з': 18, 'ч': 19, 'й': 20, 'м': 21, 'г': 22, 'б': 23, 'я': 24, 'ш': 25, 'ю': 26, 'х': 27}


In [298]:
inp_chars = 6 #
data = tokenizer.texts_to_matrix(text)

In [299]:
data[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [300]:
n = data.shape[0]-inp_chars
n  #размер обучающего множества

199

In [301]:
import numpy as np

In [302]:
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа

In [303]:
len(X)

199

In [304]:
X[0]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [305]:
Y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [306]:
from keras.layers import *
from keras.models import Sequential

In [307]:
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(500, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_10 (SimpleRNN)   (None, 500)               267500    
                                                                 
 dense_10 (Dense)            (None, 34)                17034     
                                                                 
Total params: 284,534
Trainable params: 284,534
Non-trainable params: 0
_________________________________________________________________


Создаём callback EarlyStopping (защита от переобучения)


In [308]:
erly_stoping_callback = EarlyStopping(monitor='accuracy', patience=3)

In [309]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=100,  callbacks=[erly_stoping_callback])

Epoch 1/100


7/7 [==============================] - 1s 12ms/step - loss: 3.3804 - accuracy: 0.0905
Epoch 2/100
7/7 [==============================] - 0s 11ms/step - loss: 2.4144 - accuracy: 0.3166
Epoch 3/100
7/7 [==============================] - 0s 11ms/step - loss: 1.9446 - accuracy: 0.4874
Epoch 4/100
7/7 [==============================] - 0s 11ms/step - loss: 1.5760 - accuracy: 0.5427
Epoch 5/100
7/7 [==============================] - 0s 11ms/step - loss: 1.2597 - accuracy: 0.6131
Epoch 6/100
7/7 [==============================] - 0s 11ms/step - loss: 1.0607 - accuracy: 0.6784
Epoch 7/100
7/7 [==============================] - 0s 11ms/step - loss: 0.8384 - accuracy: 0.8090
Epoch 8/100
7/7 [==============================] - 0s 12ms/step - loss: 0.7162 - accuracy: 0.8241
Epoch 9/100
7/7 [==============================] - 0s 11ms/step - loss: 0.6156 - accuracy: 0.8342
Epoch 10/100
7/7 [==============================] - 0s 11ms/step - loss: 0.5891 - accuracy: 0.8844
Epoch 11/100
7/7 [=============

In [310]:
if erly_stoping_callback.stopped_epoch >=1:
    print('Обучение остановлено на эпохе', erly_stoping_callback.stopped_epoch)

Обучение остановлено на эпохе 16


In [311]:
def buildPhrase(inp_str, str_len = 50):
  for i in range(str_len):
    x = []
    for j in range(i, i+inp_chars):
      x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-encoding

    x = np.array(x)
    inp = x.reshape(1, inp_chars, num_characters)

    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    d = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении

    inp_str += d # дописываем строку

  return inp_str

In [312]:
res = buildPhrase("утренн")


1/1 [==============================] - 0s 24ms/step


In [313]:
print(res)

утренноитввд саогя сл тс еоьтеиа евосли в  онюеил зоыйчи


# Слова


In [314]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') # убираем первый невидимый символ

In [315]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                       lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [316]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('вы', 3), ('лучший', 1), ('ответ', 1), ('на', 1), ('проблемы', 1), ('которые', 1), ('возникли', 1), ('в', 3), ('понедельник', 2), ('думайте', 1)]


In [317]:
data = tokenizer.texts_to_sequences([texts])

In [318]:
data

[[1,
  4,
  5,
  6,
  7,
  8,
  9,
  2,
  3,
  10,
  11,
  12,
  13,
  2,
  14,
  15,
  16,
  17,
  18,
  19,
  1,
  20,
  2,
  3,
  21,
  22,
  23,
  24,
  1,
  25,
  26]]

In [319]:
len(data[0])

31

In [320]:
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )

(31, 1000)


In [321]:
inp_words = 3
n = res.shape[0]-inp_words

In [322]:
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [323]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_11 (SimpleRNN)   (None, 128)               144512    
                                                                 
 dense_11 (Dense)            (None, 1000)              129000    
                                                                 
Total params: 273,512
Trainable params: 273,512
Non-trainable params: 0
_________________________________________________________________


In [324]:
history = model.fit(X, Y, batch_size=32, epochs=19, callbacks=[erly_stoping_callback])

Epoch 1/19


1/1 [==============================] - 1s 1s/step - loss: 6.9038 - accuracy: 0.0000e+00
Epoch 2/19
1/1 [==============================] - 0s 7ms/step - loss: 6.8778 - accuracy: 0.0000e+00
Epoch 3/19
1/1 [==============================] - 0s 8ms/step - loss: 6.8516 - accuracy: 0.0357
Epoch 4/19
1/1 [==============================] - 0s 12ms/step - loss: 6.8248 - accuracy: 0.2857
Epoch 5/19
1/1 [==============================] - 0s 9ms/step - loss: 6.7974 - accuracy: 0.4643
Epoch 6/19
1/1 [==============================] - 0s 8ms/step - loss: 6.7688 - accuracy: 0.6071
Epoch 7/19
1/1 [==============================] - 0s 11ms/step - loss: 6.7388 - accuracy: 0.7857
Epoch 8/19
1/1 [==============================] - 0s 8ms/step - loss: 6.7072 - accuracy: 0.8929
Epoch 9/19
1/1 [==============================] - 0s 8ms/step - loss: 6.6734 - accuracy: 1.0000
Epoch 10/19
1/1 [==============================] - 0s 11ms/step - loss: 6.6372 - accuracy: 1.0000
Epoch 11/19
1/1 [=======================

In [325]:
# Смотрим на какой эпохе остановилось обучение
if erly_stoping_callback.stopped_epoch >=1:
    print('Обучение остановлено на эпохе', erly_stoping_callback.stopped_epoch)

Обучение остановлено на эпохе 11


In [331]:
def buildPhrase(texts, str_len = 20):
  res = texts
  data = tokenizer.texts_to_sequences([texts])[0]

  for i in range(str_len):
    x = to_categorical(data[i: i+inp_words], num_classes=maxWordsCount) 
    inp = x.reshape(1, inp_words, maxWordsCount)
    pred = model.predict( inp ) 
    indx = pred.argmax(axis=1)[0]
    data.append(indx)
 
    res += " " + tokenizer.index_word[indx] 
 
  return res

In [332]:
res = buildPhrase('лучший ответ на')

1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 20ms/step


In [333]:
print(res)

лучший ответ на проблемы которые возникли в понедельник думайте позитивно и верьте в свою способность достигать отличных результатов если вы смогли в понедельник
